In [14]:
# S3 prefix
BUCKET = '{BUCKET}'
PREFIX = f"s3://{BUCKET}/mlproject/training-dataset/heart_diseases_normalizedml_withtarget_dataset.csv"

import sagemaker
import os
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session(default_bucket=BUCKET )

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()


In [15]:
from sagemaker.sklearn.estimator import SKLearn

script_path = 'deployment.py'

sklearn = SKLearn(
    entry_point=script_path,
    train_instance_type="ml.m5.large",
    framework_version='0.23-1',
    role=role,
    sagemaker_session=sagemaker_session)

sklearn.fit({'train' : PREFIX})

2020-10-08 15:16:28 Starting - Starting the training job...
2020-10-08 15:16:33 Starting - Launching requested ML instances......
2020-10-08 15:17:41 Starting - Preparing the instances for training......
2020-10-08 15:18:57 Downloading - Downloading input data
2020-10-08 15:18:57 Training - Downloading the training image......
2020-10-08 15:19:54 Uploading - Uploading generated training model2020-10-08 15:19:48,648 sagemaker-training-toolkit INFO     Imported framework sagemaker_sklearn_container.training
2020-10-08 15:19:48,650 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-10-08 15:19:48,660 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-10-08 15:19:48,981 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-10-08 15:19:52,009 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-10-08 15:19:52,020 sagemaker-training-toolkit INFO     No GPUs

In [16]:
import numpy
from sagemaker.sklearn.estimator import SKLearn,SKLearnModel


endpoint_name = 'mlr-scikit-endpoint'
predictor = sklearn.deploy(instance_type="ml.c4.xlarge", initial_instance_count=1,endpoint_name=endpoint_name)

-------------------!

In [20]:
import boto3 
import json

client = boto3.client('sagemaker-runtime')


response = client.invoke_endpoint(
    EndpointName=endpoint_name,  
#     CustomAttributes=custom_attributes, 
    ContentType='text/csv',
#     Accept=accept,
    Body= '0.2718245 , 1.74895831 , 1.79450403 ,-0.56987194 ,-1.02257128 , 1.00782704,0.2308716,0.34360555,0.01518251,-0.8005316,-0.11939036'
    )

result = response['Body']
model_prediction = json.loads(result.read())
print(model_prediction)
print(model_prediction['instances'][0]['features'])


{'instances': [{'features': 1}]}
1
